# Test surrogate augmentation on InceptionTimeClassifier


In [9]:
# make some imports 
import warnings
warnings.filterwarnings("ignore")  # hide some annoying deprication warnings

import numpy as np
import pandas as pd

import random

random.seed(11)

In [2]:
# Load data

from sktime.datasets import load_UCR_UEA_dataset
from sklearn.metrics import accuracy_score

dataset = "BME"
X_train, y_train = load_UCR_UEA_dataset(name=dataset, split="train", return_X_y=True, return_type="numpy2d")
X_test, y_test = load_UCR_UEA_dataset(name=dataset, split="test", return_X_y=True, return_type="numpy2d")

print("Training data shape is", X_train.shape)


Training data shape is (30, 128)


In [15]:
from sktime.classification.deep_learning import InceptionTimeClassifier

# Train InceptionTimeClassifier on normal data
N_EPOCHS = 15
BATCH_SIZE = 32
ITC = InceptionTimeClassifier(n_epochs=N_EPOCHS, verbose=1, batch_size=BATCH_SIZE)
ITC.fit(X_train, y_train)


# Predict and evaluate on test set
y_preditions = ITC.predict(X_test)

accuracy_score(y_test, y_preditions)

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 128, 1)]     0           []                               
                                                                                                  
 max_pooling1d_48 (MaxPooling1D  (None, 128, 1)      0           ['input_9[0][0]']                
 )                                                                                                
                                                                                                  
 conv1d_248 (Conv1D)            (None, 128, 32)      1280        ['input_9[0][0]']                
                                                                                                  
 conv1d_249 (Conv1D)            (None, 128, 32)      640         ['input_9[0][0]']          

0.5733333333333334

## Test on Augmented data 

In [10]:
# Create a subset of the training data to be augmented

from copy import deepcopy

N = X_train.shape[0]
p = 0.2     # percentage of data to augment
rows_to_augment = random.sample(range(0,N), int(N*p))

X_subset = deepcopy(X_test[rows_to_augment,:])
y_subset = deepcopy(y_train[rows_to_augment])



In [11]:
from surrogate_augmentation import correlated_noise_surrogates, AAFT_surrogates, refined_AAFT_surrogates

# Correlated noise surrogate
X_subset_augmented = correlated_noise_surrogates(X_subset)
X_train_augmented = np.concatenate((X_train, X_subset_augmented))
y_train_augmented = np.concatenate((y_train, y_subset))


ITC2 = InceptionTimeClassifier(n_epochs=N_EPOCHS, verbose=1, batch_size=BATCH_SIZE)
ITC2.fit(X_train_augmented, y_train_augmented)
y_preditions = ITC2.predict(X_test)
accuracy_score(y_test, y_preditions)


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 128, 1)]     0           []                               
                                                                                                  
 max_pooling1d_24 (MaxPooling1D  (None, 128, 1)      0           ['input_5[0][0]']                
 )                                                                                                
                                                                                                  
 conv1d_124 (Conv1D)            (None, 128, 32)      1280        ['input_5[0][0]']                
                                                                                                  
 conv1d_125 (Conv1D)            (None, 128, 32)      640         ['input_5[0][0]']          

0.7533333333333333

In [12]:
# AFFT surrogates

X_subset_augmented2 = AAFT_surrogates(X_subset)
X_train_augmented2 = np.concatenate((X_train, X_subset_augmented2))

ITC3 = InceptionTimeClassifier(n_epochs=N_EPOCHS, verbose=1, batch_size=BATCH_SIZE)
ITC3.fit(X_train_augmented2, y_train_augmented)
y_preditions = ITC3.predict(X_test)
accuracy_score(y_test, y_preditions)


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 128, 1)]     0           []                               
                                                                                                  
 max_pooling1d_30 (MaxPooling1D  (None, 128, 1)      0           ['input_6[0][0]']                
 )                                                                                                
                                                                                                  
 conv1d_155 (Conv1D)            (None, 128, 32)      1280        ['input_6[0][0]']                
                                                                                                  
 conv1d_156 (Conv1D)            (None, 128, 32)      640         ['input_6[0][0]']          

0.84

In [14]:
# AAFT refined surrogates
n_iters = 10

X_subset_augmented3 = refined_AAFT_surrogates(X_subset, n_iterations=n_iters)
X_train_augmented3 = np.concatenate((X_train, X_subset_augmented3), axis=0)

ITC4 = InceptionTimeClassifier(n_epochs=N_EPOCHS, verbose=1, batch_size=BATCH_SIZE)
ITC4.fit(X_train_augmented3, y_train_augmented)
y_preditions = ITC4.predict(X_test)
accuracy_score(y_test, y_preditions)




Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 128, 1)]     0           []                               
                                                                                                  
 max_pooling1d_42 (MaxPooling1D  (None, 128, 1)      0           ['input_8[0][0]']                
 )                                                                                                
                                                                                                  
 conv1d_217 (Conv1D)            (None, 128, 32)      1280        ['input_8[0][0]']                
                                                                                                  
 conv1d_218 (Conv1D)            (None, 128, 32)      640         ['input_8[0][0]']          

0.76